# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 21 2023 8:55AM,257806,10,9186928,SugarBear,Released
1,Feb 21 2023 8:45AM,257804,12,HH-40051,Hush Hush,Released
2,Feb 21 2023 8:45AM,257804,12,HH-40052,Hush Hush,Released
3,Feb 21 2023 8:45AM,257804,12,HH-40053,Hush Hush,Released
4,Feb 21 2023 8:45AM,257804,12,HH-40054,Hush Hush,Released
5,Feb 21 2023 8:45AM,257804,12,HH-40055,Hush Hush,Released
6,Feb 21 2023 8:45AM,257804,12,HH-40056,Hush Hush,Released
7,Feb 21 2023 8:45AM,257804,12,HH-40068,Hush Hush,Released
8,Feb 21 2023 8:45AM,257804,12,HH-40089,Hush Hush,Released
9,Feb 21 2023 8:45AM,257804,12,HH-40083,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,257802,Released,16
33,257803,Released,6
34,257804,Released,38
35,257805,Executing,1
36,257806,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257802,NaN,NaN,16.0
257803,NaN,NaN,6.0
257804,NaN,NaN,38.0
257805,NaN,1.0,NaN
257806,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257705,1.0,10.0,11.0
257711,0.0,1.0,52.0
257713,0.0,0.0,37.0
257715,0.0,0.0,56.0
257719,0.0,0.0,62.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257705,1,10,11
257711,0,1,52
257713,0,0,37
257715,0,0,56
257719,0,0,62


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257705,1,10,11
1,257711,0,1,52
2,257713,0,0,37
3,257715,0,0,56
4,257719,0,0,62


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257705,1,10,11
1,257711,,1,52
2,257713,,,37
3,257715,,,56
4,257719,,,62


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 21 2023 8:55AM,257806,10,SugarBear
1,Feb 21 2023 8:45AM,257804,12,Hush Hush
39,Feb 21 2023 8:44AM,257805,12,Hush Hush
40,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated"
41,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation
42,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation
109,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation
111,Feb 21 2023 8:29AM,257803,10,Emerginnova
117,Feb 21 2023 8:25AM,257802,10,ISDIN Corporation
133,Feb 21 2023 8:20AM,257800,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 21 2023 8:55AM,257806,10,SugarBear,,,1
1,Feb 21 2023 8:45AM,257804,12,Hush Hush,,,38
2,Feb 21 2023 8:44AM,257805,12,Hush Hush,,1,
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",,,1
4,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,,,58
5,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,,,10
6,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,,,2
7,Feb 21 2023 8:29AM,257803,10,Emerginnova,,,6
8,Feb 21 2023 8:25AM,257802,10,ISDIN Corporation,,,16
9,Feb 21 2023 8:20AM,257800,10,ISDIN Corporation,,,45


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:55AM,257806,10,SugarBear,1,,
1,Feb 21 2023 8:45AM,257804,12,Hush Hush,38,,
2,Feb 21 2023 8:44AM,257805,12,Hush Hush,,1,
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1,,
4,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58,,
5,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,10,,
6,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,2,,
7,Feb 21 2023 8:29AM,257803,10,Emerginnova,6,,
8,Feb 21 2023 8:25AM,257802,10,ISDIN Corporation,16,,
9,Feb 21 2023 8:20AM,257800,10,ISDIN Corporation,45,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:55AM,257806,10,SugarBear,1,,
1,Feb 21 2023 8:45AM,257804,12,Hush Hush,38,,
2,Feb 21 2023 8:44AM,257805,12,Hush Hush,,1,
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1,,
4,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:55AM,257806,10,SugarBear,1.0,NaN,NaN
1,Feb 21 2023 8:45AM,257804,12,Hush Hush,38.0,NaN,NaN
2,Feb 21 2023 8:44AM,257805,12,Hush Hush,NaN,1.0,NaN
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:55AM,257806,10,SugarBear,1.0,0.0,0.0
1,Feb 21 2023 8:45AM,257804,12,Hush Hush,38.0,0.0,0.0
2,Feb 21 2023 8:44AM,257805,12,Hush Hush,0.0,1.0,0.0
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6701750,547.0,11.0,1.0
12,515609,38.0,1.0,0.0
16,515532,1.0,1.0,0.0
19,515595,18.0,5.0,0.0
22,257792,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6701750,547.0,11.0,1.0
1,12,515609,38.0,1.0,0.0
2,16,515532,1.0,1.0,0.0
3,19,515595,18.0,5.0,0.0
4,22,257792,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,547.0,11.0,1.0
1,12,38.0,1.0,0.0
2,16,1.0,1.0,0.0
3,19,18.0,5.0,0.0
4,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,547.0
1,12,Released,38.0
2,16,Released,1.0
3,19,Released,18.0
4,22,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,22
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,11.0,1.0,1.0,5.0,1.0
Released,547.0,38.0,1.0,18.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,22
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,11.0,1.0,1.0,5.0,1.0
2,Released,547.0,38.0,1.0,18.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,22
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,11.0,1.0,1.0,5.0,1.0
2,Released,547.0,38.0,1.0,18.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()